In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import random
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import cv2
import urllib.request as req

In [2]:
ndata = pd.read_csv('USvideos.csv').loc[:, ["trending_date","title","channel_title","category_id","publish_time","tags","views","likes","dislikes","comment_count","thumbnail_link","comments_disabled","ratings_disabled","video_error_or_removed","description"]]

lnks = ndata.loc[:, ["title","thumbnail_link", "views"]].drop_duplicates(subset = "thumbnail_link", keep = "last").reset_index(drop = True)

In [3]:
# x = []
# z = []
# c = 0

# for i in lnks["thumbnail_link"]:
    
#     if c <= 5000:
        
#         try:
#             req.urlretrieve(i, "{}.jpg".format(c))
#             x.append(lnks["views"][lnks.thumbnail_link[lnks.thumbnail_link == i].index.tolist()[0]])
#             z.append(lnks["title"][lnks.thumbnail_link[lnks.thumbnail_link == i].index.tolist()[0]])
#             c += 1
            
#         except:
#             pass
    
#     else:
#         break
    
# print(c)

In [4]:
# vectorizer = CountVectorizer()
# vectx = vectorizer.fit_transform(z)
# vectx = vectx.toarray()

In [5]:
# with open('z.txt', 'w') as f:
#     for item in vectx:
#         for i in item:
#             f.write(str(i) + "\n")

# with open('x.txt', 'w') as f:
#     for item in x:
#         f.write(str(item) + "\n")
        
# len(vectx[0])

---

In [22]:
# faixa 1 = 0, 100652.364
# faixa 2 = 100652.364, 504283.8
# faixa 3 = 504283.8, 1008510.6
# faixa 4 = 1008510.6, 5127377.4
# faixa 5 = 5127377.4, 50169650.2
# faixa 6 = 50169650.2, 225211923.0

views_y = []

with open('x.txt', 'r') as f:
    for item in f:
        if int(item[:-1]) < 100000:
            views_y.append(1)
        
        if int(item[:-1]) > 100000 and int(item[:-1]) < 500000:
            views_y.append(2)
        
        if int(item[:-1]) > 500000 and int(item[:-1]) < 1000000:
            views_y.append(3)
        
        if int(item[:-1]) > 1000000 and int(item[:-1]) < 5000000:
            views_y.append(4)
        
        if int(item[:-1]) > 5000000:
            views_y.append(5)

len(vect)

5000

In [16]:
c = 0
vect = []
l = []

with open('z.txt', 'r') as f:
    for item in f:
        
        if c < 9115:
            l.append(int(item[:-1]))
            
        else:
            vect.append(l)
            l = []
            c = 0
            
        c += 1      

In [8]:
lista_x = []

for i in range(5001):
    im = cv2.imread('{}.jpg'.format(i))
    img_crop = cv2.pyrDown(im)
    img_crop = cv2.pyrDown(img_crop)
    img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
    lista_x.append(img_crop.flatten())
    
lista_x = np.concatenate((lista_x, vect),axis=1)

ValueError: all the input arrays must have same number of dimensions

---

In [ ]:
shuff = list(zip(lista_x, views_y))

random.shuffle(shuff)

lista_x, views_y = zip(*shuff)

In [ ]:
index = round(len(lista_x)*0.95)
lista_x_train, lista_x_test = lista_x[:index], lista_x[index:]
len(lista_x_train)

In [ ]:
views_y
index_ = round(len(views_y)*0.95)
views_y_train, views_y_test = views_y[:index_], views_y[index_:]
len(views_y_train)

---

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss = "hinge", penalty = "l2", max_iter = 100)
clf.fit(lista_x_train, views_y_train)
y_pred = clf.predict(lista_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, max_depth = 2,random_state = 0)
clf.fit(lista_x_train, views_y_train)
y_pred = clf.predict(lista_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

In [ ]:
from sklearn.kernel_approximation import RBFSampler

rbf_feature = RBFSampler(gamma = 1, random_state = 1)
X_features = rbf_feature.fit_transform(lista_x_train)
clf = SGDClassifier(max_iter = 100)
clf.fit(lista_x_train, views_y_train) 
y_pred = clf.predict(lista_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

---

In [ ]:
y_pred

### Análise Exploratória

O Objetivo do projeto é indentificar em qual faixa de views (0 - 100 mil, ...) um vídeo vai estar, dada somente sua tumbnail como entrada. Para isso, foi baixado um dataset contendo diversas variáveis como "likes", "dislikes", ..., etc. Cada coluna diz respeito a algum feature que o youtube utiliza como caracteristicas do video, como pode ser visto abaixo:

In [ ]:
ndata

Como o nosso projeto consta somente com a imagem, cortamos o dataset em um com apenas o link das tumbnails e o número de views do vídeo. É importante dizer que os dados foram atualizados todos os dias durante o tempo, o que significa que existem videos iguais em diferentes dias, e por isso utilizamos do ferramental pandas para tirar todas as tumbnails iguais. Por conta disso, em alguns casos o número de views é diferente para o mesmo link (vídeo), optamos por considerar a ultima vez q o dataset foi atualizado, ou seja, o maior número de views de um vídeo igual.

In [ ]:
lnks

Com o banco de dados pronto para ser utilizado, o próximo passo seria baixar as fotos para o computador, para otimizar o tempo na hora de trabalhar no modelo. Como muitas tumbnails não existem (vídeo removido, não existe mais foto), foi utilizado o try, e logo depois adicionado o número de views a uma lista chamada "x" (não tem relação com o modelo, o nome "x" é apenas representativo). 

    import urllib.request as req

    x = []
    c = 0

    for i in lnks["thumbnail_link"]:

        if c <= 5000:

            try:
                req.urlretrieve(i, "{}.jpg".format(c))
                x.append(lnks["views"][lnks.thumbnail_link[lnks.thumbnail_link == i].index.tolist()[0]])
                c += 1

            except:
                pass

        else:
            break

    print(c)

Além disso, como nosso modelo adota o uso de imagens para análise do numero de views, não é possível plotar histogramas ou boxplots, o que nos leva a descartar tal tipo de análise.

Foi salvo os views da lista "x" em um arquivo para nao ter de rodar novamente o código acima, o que acabaria baixando novamente as fotos.

    with open('x.txt', 'w') as f:
        for item in x:
            f.write(str(item) + "\n")

Com o y pronto (número de views) e dividido em faixas, o próximo passo seria achar algum meio de transformar as imagens baixadas em um array que representaria o x do modelo.



Para isso foi utilizado o cv2 para ler e converter as imagens em array, e prontas para serem utilizadas no modelo.

    import numpy as np
    import cv2

    imagens_x = []

    for i in range(5001):
        im = cv2.imread('{}.jpg'.format(i))
        img_crop = cv2.pyrDown(im)
        img_crop = cv2.pyrDown(img_crop)
        img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
        imagens_x.append(img_crop.flatten())

Antes de testar os modelos, algumas alterações foram feitas nas listas x e y, como por exemplo o shuffle. Dividimos 95% das duas listas para treino, e o resto para classificação.
Ao entrar no site do sikit em "Flow Chart", alguns modelos foram testados seguindo a sugestão do mesmo.
Como o máximo de acurácia foi 33%, escolhemos entender melhor o que as imagens nos diziam, portanto, o proximo passo para melhorar a acurácia é criar meios de análise da imagem, para ajudar o classificador a melhorar os acertos.